In [1]:
import carla
import argparse
import time
import signal
import sys
import traceback
import os
import subprocess
import re
import socket
import atexit
import threading
from datetime import datetime
from xml.etree import ElementTree as ET

# os.environ['CUDA_HOME'] = '/usr/local/cuda-12.2'
os.environ['IS_BENCH2DRIVE'] = 'true'
os.environ['REPETITION'] = '1'
# os.environ['DATAGEN'] = '1'


In [2]:
import subprocess, re

def kill_carla_processes_on_gpu(gpu_index: int = 0):
    """Kill CARLA processes on specific GPU"""
    try:
        raw = subprocess.check_output("nvidia-smi", shell=True).decode()
        pattern = rf'\|\s+{gpu_index}\s+\S+\s+\S+\s+(\d+)\s+C\+G\s+.*CarlaUE4'
        pids = re.findall(pattern, raw)
        if not pids:
            print(f"No CarlaUE4 processes found on GPU {gpu_index}")
            return
        for pid in pids:
            print(f"Killing CarlaUE4 process with PID {pid} on GPU {gpu_index}")
            subprocess.run(["kill", "-9", pid], check=True)
    except Exception as e:
        print(f"Error killing CARLA processes: {e}")

def kill_python_processes_on_gpu(gpu_index: int = 0):
    """Kill Python processes on specific GPU"""
    try:
        raw = subprocess.check_output("nvidia-smi", shell=True).decode()
        pattern = rf'\|\s+{gpu_index}\s+\S+\s+\S+\s+(\d+)\s+C\s+.*bin/python'
        pids = re.findall(pattern, raw)
        if not pids:
            print(f"No python processes found on GPU {gpu_index}")
            return
        for pid in pids:
            print(f"Killing python process with PID {pid} on GPU {gpu_index}")
            subprocess.run(["kill", "-9", pid], check=True)
    except Exception as e:
        print(f"Error killing Python processes: {e}")

def cleanup_gpu_processes():
    """Clean up GPU processes"""
    for gpu_id in [0, 1]:
        kill_carla_processes_on_gpu(gpu_id)
        kill_python_processes_on_gpu(gpu_id)

cleanup_gpu_processes()

Killing CarlaUE4 process with PID 2308331 on GPU 0
No python processes found on GPU 0
No CarlaUE4 processes found on GPU 1
No python processes found on GPU 1


In [3]:
# Setup paths
leaderboard_path = '/mnt3/Documents/AD_Framework/Bench2Drive/leaderboard/leaderboard'
if leaderboard_path not in sys.path:
    sys.path.insert(0, leaderboard_path)

# Check environment variables
print("CARLA_ROOT: ", os.environ.get('CARLA_ROOT'))
print("WORK_DIR: ", os.environ.get('WORK_DIR'))
print("SCENARIO_RUNNER_ROOT: ", os.environ.get('SCENARIO_RUNNER_ROOT'))
print("LD_LIBRARY_PATH: ", os.environ.get('LD_LIBRARY_PATH'))
print("LEADERBOARD_ROOT: ", os.environ.get('LEADERBOARD_ROOT'))
print("CUDA_HOME: ", os.environ.get('CUDA_HOME'))
print("PYTHONPATH: ", os.environ.get('PYTHONPATH'))

CARLA_ROOT:  /mnt3/Documents/AD_Framework/carla0915
WORK_DIR:  /mnt3/Documents/AD_Framework/Bench2Drive
SCENARIO_RUNNER_ROOT:  /mnt3/Documents/AD_Framework/Bench2Drive/scenario_runner
LD_LIBRARY_PATH:  /usr/local/cuda-12.1/lib64:
LEADERBOARD_ROOT:  /mnt3/Documents/AD_Framework/Bench2Drive/leaderboard
CUDA_HOME:  /usr/local/cuda-12.1
PYTHONPATH:  /mnt3/Documents/AD_Framework/carla0915/PythonAPI/carla/:/mnt3/Documents/AD_Framework/Bench2Drive/scenario_runner:/mnt3/Documents/AD_Framework/Bench2Drive/leaderboard::/mnt3/Documents/AD_Framework/Bench2Drive:/mnt3/Documents/AD_Framework/carla0915/PythonAPI:/mnt3/Documents/AD_Framework/carla0915/PythonAPI/carla:/mnt3/Documents/AD_Framework/carla0915/PythonAPI/carla/:/mnt3/Documents/AD_Framework/Bench2Drive/scenario_runner:/mnt3/Documents/AD_Framework/Bench2Drive/leaderboard::/mnt3/Documents/AD_Framework/Bench2Drive:/mnt3/Documents/AD_Framework/carla0915/PythonAPI:/mnt3/Documents/AD_Framework/carla0915/PythonAPI/carla


In [4]:
port = 20019

model_name = 'my_model_mlp'
statistics_manager_path = f'/mnt3/Documents/AD_Framework/results/{model_name}'

# makedir if do not exist
os.makedirs(statistics_manager_path, exist_ok=True)

args = argparse.Namespace()
args.WORK_DIR = os.environ.get('WORK_DIR')
args.checkpoint = f'{statistics_manager_path}/stat_manager_results.json'
args.debug_checkpoint = f'{statistics_manager_path}/live_results.txt'
args.host = '127.0.0.3'
args.port = port
args.timeout = 6000
args.frame_rate = 20.0 # fixed
args.gpu_rank = 0
args.traffic_manager_port = port + 3
# '/po3/korawat/Documents/AD_Framework/Bench2Drive/leaderboard/leaderboard/autoagents/dummy_agent.py'
# args.TEAM_AGENT = f"{args.WORK_DIR}/leaderboard/team_code/data_agent2.py" # auto_pilot.py
args.TEAM_AGENT = f'{leaderboard_path}/autoagents/dummy_agent4.py'
## auto_pilot.py
args.agent = args.TEAM_AGENT
print("args.agent", args.agent)
args.agent_config = ''
args.debug = 1
args.routes = f"/mnt3/Documents/AD_Framework/Bench2Drive/leaderboard/data/bench2drive220.xml"
args.repetitions = 1
args.track = 'SENSORS' # Changed from 'MAP' to 'SENSORS' to allow camera sensors
args.record = ''
args.routes_subset = 0
args.resume = 0
args.traffic_manager_seed = port - 3333 + 1

# Enable camera sensors by setting required environment variables
os.environ['SAVE_PATH'] = statistics_manager_path
os.environ['IS_BENCH2DRIVE'] = 'true'
os.environ['TMP_VISU'] = '0'  # Enable camera sensors for visualization

print(f"SAVE_PATH set to: {statistics_manager_path}")
print(f"TMP_VISU set to: {os.environ.get('TMP_VISU')}")
print(f"Track set to: {args.track}")

args.agent /mnt3/Documents/AD_Framework/Bench2Drive/leaderboard/leaderboard/autoagents/dummy_agent4.py
SAVE_PATH set to: /mnt3/Documents/AD_Framework/results/my_model_mlp
TMP_VISU set to: 0
Track set to: SENSORS


In [5]:
from srunner.scenariomanager.carla_data_provider import *
from srunner.scenariomanager.timer import GameTime
from srunner.scenariomanager.watchdog import Watchdog

from leaderboard.scenarios.scenario_manager import ScenarioManager
from leaderboard.scenarios.route_scenario import RouteScenario
from leaderboard.envs.sensor_interface import SensorConfigurationInvalid
from leaderboard.autoagents.agent_wrapper import AgentError, validate_sensor_configuration, TickRuntimeError
from leaderboard.utils.statistics_manager import StatisticsManager, FAILURE_MESSAGES
from leaderboard.utils.route_indexer import RouteIndexer

# print("args: ", args)
statistics_manager = StatisticsManager(args.checkpoint , args.debug_checkpoint)
# statistics_manager.write_statistics() ## run one time

# leaderboard_evaluator = LeaderboardEvaluator(args, statistics_manager)

LeaderboardEvaluator_path = '/po3/korawat/Documents/AD_Framework/Bench2Drive/leaderboard/leaderboard/leaderboard_evaluator.py'
if LeaderboardEvaluator_path not in sys.path:
    sys.path.insert(0, LeaderboardEvaluator_path)

from leaderboard.leaderboard_evaluator import LeaderboardEvaluator


In [6]:
import socket
import subprocess
import atexit


def find_free_port(starting_port):
    """Find a free port starting from the given port"""
    port = starting_port
    while True:
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(("localhost", port))
                return port
        except OSError:
            port += 1

def _setup_simulation(self, args):
    """Setup simulation with CARLA server"""
    os.system("pkill -f CarlaUE4")
    time.sleep(5)

    self.carla_path = os.environ["CARLA_ROOT"]
    args.port = find_free_port(args.port)
    cmd1 = f"{os.path.join(self.carla_path, 'CarlaUE4.sh')} -RenderOffScreen -nosound -quality-level=Epic -carla-rpc-port={args.port} -carla-streaming-port=0 -graphicsadapter=0"
    self.server = subprocess.Popen(cmd1, shell=True, preexec_fn=os.setsid)
    atexit.register(os.killpg, self.server.pid, signal.SIGKILL)

    # Wait for server to be ready
    server_ready = False
    for _ in range(30):
        try:
            with socket.create_connection(('localhost', args.port), timeout=1):
                server_ready = True
                break
        except (ConnectionRefusedError, socket.timeout):
            time.sleep(5)
    if not server_ready:
        raise RuntimeError("Carla server failed to start")
        
    attempts = 0
    num_max_restarts = 20
    print("Loading world!!!")
    while attempts < num_max_restarts:
        try:
            client = carla.Client(args.host, args.port)
            if args.timeout:
                client_timeout = args.timeout
            client.set_timeout(client_timeout)

            settings = carla.WorldSettings(
                synchronous_mode = True,
                fixed_delta_seconds = 1.0 / self.frame_rate,
                deterministic_ragdolls = True,
                spectator_as_ego = False
            )
            client.get_world().apply_settings(settings)
            print(f"load_world success , attempts={attempts}", flush=True)
            break
        except Exception as e:
            print(f"load_world failed , attempts={attempts}", flush=True)
            print(e, flush=True)
            attempts += 1
            time.sleep(5)
    
    attempts = 0
    num_max_restarts = 40
    while attempts < num_max_restarts:
        try:
            args.traffic_manager_port = find_free_port(args.traffic_manager_port)
            traffic_manager = client.get_trafficmanager(args.traffic_manager_port)
            traffic_manager.set_synchronous_mode(True)
            traffic_manager.set_hybrid_physics_mode(True)
            print(f"traffic_manager init success, try_time={attempts}", flush=True)
            break
        except Exception as e:
            print(f"traffic_manager init fail, try_time={attempts}", flush=True)
            print(e, flush=True)
            attempts += 1
            time.sleep(5)

    return client, client_timeout, traffic_manager


def get_weather_id(weather_conditions):
    from xml.etree import ElementTree as ET
    WORK_DIR = os.environ.get("WORK_DIR", "")
    if WORK_DIR != "":
        WORK_DIR += "/"
    tree = ET.parse(WORK_DIR + 'leaderboard/data/weather.xml')
    root = tree.getroot()
    def conditions_match(weather, conditions):
        for (key, value) in weather:
            if key == 'route_percentage' : continue
            if str(getattr(conditions, key))!= value:
                return False
        return True
    for case in root.findall('case'):
        weather = case[0].items()
        if conditions_match(weather, weather_conditions):
            return case.items()[0][1]
    return None

sensors_to_icons = {
    'sensor.camera.rgb':        'carla_camera',
    'sensor.lidar.ray_cast':    'carla_lidar',
    'sensor.other.radar':       'carla_radar',
    'sensor.other.gnss':        'carla_gnss',
    'sensor.other.imu':         'carla_imu',
    'sensor.opendrive_map':     'carla_opendrive_map',
    'sensor.speedometer':       'carla_speedometer'
}

#### the following are the functions that equal to _load_and_run_scenario in leaderboard_evaluator.py

def my_load_scenario(leaderboard_evaluator_self, args, config):
    """Load and run the scenario given by config"""
    crash_message = ""
    entry_status = "Started"
    save_name = ""

    print("\n\033[1m========= Preparing {} (repetition {}) =========\033[0m".format(config.name, config.repetition_index), flush=True)

    # Prepare the statistics of the route
    route_name = f"{config.name}_rep{config.repetition_index}"
    scenario_name = config.scenario_configs[0].name
    town_name = str(config.town)
    weather_id = get_weather_id(config.weather[0][1])
    currentDateAndTime = datetime.now()
    currentTime = currentDateAndTime.strftime("%m_%d_%H_%M_%S")
    save_name = f"{route_name}_{town_name}_{scenario_name}_{weather_id}_{currentTime}"
    leaderboard_evaluator_self.statistics_manager.create_route_data(route_name, scenario_name, weather_id, save_name, town_name, config.index)

    print("\033[1m> Loading the world\033[0m", flush=True)

    # Load the world and the scenario
    try:
        leaderboard_evaluator_self._load_and_wait_for_world(args, config.town)
        from leaderboard.scenarios.route_scenario import RouteScenario
        leaderboard_evaluator_self.route_scenario = RouteScenario(world=leaderboard_evaluator_self.world, config=config, debug_mode=args.debug)
        leaderboard_evaluator_self.statistics_manager.set_scenario(leaderboard_evaluator_self.route_scenario)

    except Exception:
        # The scenario is wrong -> set the ejecution to crashed and stop
        print("\n\033[91mThe scenario could not be loaded:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m", flush=True)

        from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
        entry_status, crash_message = FAILURE_MESSAGES["Simulation"]
        leaderboard_evaluator_self._register_statistics(config.index, entry_status, crash_message)
        leaderboard_evaluator_self._cleanup()
        return True, save_name, entry_status, crash_message

    return False, save_name, entry_status, crash_message

def my_load_agent(leaderboard_evaluator_self, args, config, save_name, entry_status, crash_message):
    """Set up the user's agent and configure sensors"""
    print("\033[1m> Setting up the agent\033[0m", flush=True)

    # Set up the user's agent, and the timer to avoid freezing the simulation
    try:
        from srunner.scenariomanager.watchdog import Watchdog
        from leaderboard.envs.sensor_interface import SensorConfigurationInvalid
        from leaderboard.autoagents.agent_wrapper import validate_sensor_configuration
        from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
        
        leaderboard_evaluator_self._agent_watchdog = Watchdog(args.timeout)
        leaderboard_evaluator_self._agent_watchdog.start()
        agent_class_name = getattr(leaderboard_evaluator_self.module_agent, 'get_entry_point')()
        agent_class_obj = getattr(leaderboard_evaluator_self.module_agent, agent_class_name)

        # Start the ROS1 bridge server only for ROS1 based agents.
        if getattr(agent_class_obj, 'get_ros_version')() == 1 and leaderboard_evaluator_self._ros1_server is None:
            from leaderboard.autoagents.ros1_agent import ROS1Server
            leaderboard_evaluator_self._ros1_server = ROS1Server()
            leaderboard_evaluator_self._ros1_server.start()

        leaderboard_evaluator_self.agent_instance = agent_class_obj(args.host, args.port, args.debug)
        leaderboard_evaluator_self.agent_instance.set_global_plan(leaderboard_evaluator_self.route_scenario.gps_route, leaderboard_evaluator_self.route_scenario.route)
        args.agent_config = args.agent_config + '+' + save_name
        leaderboard_evaluator_self.agent_instance.setup(args.agent_config)

        # Check and store the sensors
        if not leaderboard_evaluator_self.sensors:
            leaderboard_evaluator_self.sensors = leaderboard_evaluator_self.agent_instance.sensors()
            track = leaderboard_evaluator_self.agent_instance.track

            validate_sensor_configuration(leaderboard_evaluator_self.sensors, track, args.track)

            leaderboard_evaluator_self.sensor_icons = [sensors_to_icons[sensor['type']] for sensor in leaderboard_evaluator_self.sensors]
            leaderboard_evaluator_self.statistics_manager.save_sensors(leaderboard_evaluator_self.sensor_icons)
            leaderboard_evaluator_self.statistics_manager.write_statistics()

            leaderboard_evaluator_self.sensors_initialized = True

        leaderboard_evaluator_self._agent_watchdog.stop()
        leaderboard_evaluator_self._agent_watchdog = None

    except SensorConfigurationInvalid as e:
        # The sensors are invalid -> set the ejecution to rejected and stop
        print("\n\033[91mThe sensor's configuration used is invalid:", flush=True)
        print(f"{e}\033[0m\n", flush=True)

        entry_status, crash_message = FAILURE_MESSAGES["Sensors"]
        leaderboard_evaluator_self._register_statistics(config.index, entry_status, crash_message)
        leaderboard_evaluator_self._cleanup()
        return True, entry_status, crash_message

    except Exception as e:
        # The agent setup has failed -> start the next route
        print("\n\033[91mCould not set up the required agent:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m", flush=True)
        print(f"{e}\033[0m\n", flush=True)

        entry_status, crash_message = FAILURE_MESSAGES["Agent_init"]
        leaderboard_evaluator_self._register_statistics(config.index, entry_status, crash_message)
        leaderboard_evaluator_self._cleanup()
        return True, entry_status, crash_message

    return False, entry_status, crash_message

def my_load_route_scenario(leaderboard_evaluator_self, args, config, entry_status, crash_message):
    """Load the route scenario and prepare for execution"""
    from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
    
    print("\033[1m> Running the route\033[0m", flush=True)

    # Run the scenario
    try:
        # Load scenario and run it
        if args.record:
            leaderboard_evaluator_self.client.start_recorder("{}/{}_rep{}.log".format(args.record, config.name, config.repetition_index))
        leaderboard_evaluator_self.manager.load_scenario(leaderboard_evaluator_self.route_scenario, leaderboard_evaluator_self.agent_instance, config.index, config.repetition_index)
    except Exception:
        print("\n\033[91mFailed to load the scenario, the statistics might be empty:", flush=True)
        print("\n\033[91mLoading the route, the agent has crashed:", flush=True)
        entry_status, crash_message = FAILURE_MESSAGES["Agent_runtime"]

    return False, entry_status, crash_message

def my_run_scenario_setup(leaderboard_evaluator_self):
    """Trigger the start of the scenario and wait for it to finish/fail"""
    from srunner.scenariomanager.timer import GameTime
    from srunner.scenariomanager.watchdog import Watchdog
    
    leaderboard_evaluator_self.manager.tick_count = 0
    leaderboard_evaluator_self.manager.start_system_time = time.time()
    leaderboard_evaluator_self.manager.start_game_time = GameTime.get_time()

    # Detects if the simulation is down
    leaderboard_evaluator_self.manager._watchdog = Watchdog(leaderboard_evaluator_self.manager._timeout)
    leaderboard_evaluator_self.manager._watchdog.start()

    # Stop the agent from freezing the simulation
    leaderboard_evaluator_self.manager._agent_watchdog = Watchdog(leaderboard_evaluator_self.manager._timeout)
    leaderboard_evaluator_self.manager._agent_watchdog.start()

    leaderboard_evaluator_self.manager._running = True

    # Thread for build_scenarios
    leaderboard_evaluator_self.manager._scenario_thread = threading.Thread(target=leaderboard_evaluator_self.manager.build_scenarios_loop, args=(leaderboard_evaluator_self.manager._debug_mode > 0, ))
    leaderboard_evaluator_self.manager._scenario_thread.start()

def my_run_scenario_step(leaderboard_evaluator_self, entry_status, crash_message, n_steps=1):
    """Run n_steps of the scenario simulation"""
    from leaderboard.autoagents.agent_wrapper import AgentError, TickRuntimeError
    from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
    
    try:
        if leaderboard_evaluator_self.manager._running:
            print("In my_run_scenario_step (Check if it is stuck or not)")
            for _ in range(n_steps):
                leaderboard_evaluator_self.manager._tick_scenario()

    except AgentError:
        # The agent has failed -> stop the route
        print("\n\033[91mStopping the route, the agent has crashed:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m")

        entry_status, crash_message = FAILURE_MESSAGES["Agent_runtime"]

        return True, entry_status, crash_message

    except KeyboardInterrupt:
        return True, entry_status, crash_message
    
    except TickRuntimeError:
        entry_status, crash_message = "Started", "TickRuntime"
        return True, entry_status, crash_message
    
    except Exception:
        print("\n\033[91mError during the simulation:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m", flush=True)

        entry_status, crash_message = FAILURE_MESSAGES["Simulation"]
        return True, entry_status, crash_message
    
    return False, entry_status, crash_message

def my_stop_scenario(leaderboard_evaluator_self, args, config, entry_status, crash_message):
    """Stop the scenario and register statistics"""
    from leaderboard.utils.statistics_manager import FAILURE_MESSAGES
    
    # Stop the scenario
    try:
        print("\033[1m> Stopping the route\033[0m", flush=True)
        leaderboard_evaluator_self.manager.stop_scenario()
        leaderboard_evaluator_self._register_statistics(config.index, entry_status, crash_message)

        if args.record:
            leaderboard_evaluator_self.client.stop_recorder()

        leaderboard_evaluator_self._cleanup()

    except Exception:
        print("\n\033[91mFailed to stop the scenario, the statistics might be empty:", flush=True)
        print(f"\n{traceback.format_exc()}\033[0m", flush=True)

        _, crash_message = FAILURE_MESSAGES["Simulation"]

    return False, entry_status, crash_message

# replace _setup_simulation method of LeaderboardEvaluator with this new one 
LeaderboardEvaluator._setup_simulation = _setup_simulation


In [7]:
### Main code object

leaderboard_evaluator = LeaderboardEvaluator(args, statistics_manager)

4.26.2-0+++UE4+Release-4.26 522 0
Disabling core dumps.
Loading world!!!
load_world success , attempts=0
traffic_manager init success, try_time=0


In [8]:
print("args.routes", args.routes)
print("args.repetitions", args.repetitions)
print("args.routes_subset", args.routes_subset)

args.routes /mnt3/Documents/AD_Framework/Bench2Drive/leaderboard/data/bench2drive220.xml
args.repetitions 1
args.routes_subset 0


In [9]:
route_indexer = RouteIndexer(args.routes, args.repetitions, args.routes_subset)
if args.resume:
    resume = route_indexer.validate_and_resume(args.checkpoint)
else:
    resume = False

if resume:
    leaderboard_evaluator.statistics_manager.add_file_records(args.checkpoint)
else:
    leaderboard_evaluator.statistics_manager.clear_records()
leaderboard_evaluator.statistics_manager.save_progress(route_indexer.index, route_indexer.total)
leaderboard_evaluator.statistics_manager.write_statistics()

In [10]:
crashed_flag = False
t1 = time.time()
route_count = 0

In [11]:
### in the loop of running

want_list = ['1790'] #['25845']

for i in range(250):
    config = route_indexer.get_next_config()
    if config is None:
        print("No more routes to process")
        

    route_id = config.name.split("_")[1]
    print("route_id: ", route_id)
    if route_id in want_list and len(want_list) > 0:
        print("Got the route_id: ", route_id)
        break
    elif len(want_list) == 0:
        route_id = route_id
        print("Got the route_id: ", route_id)
        break
    
print(f"\n=== Processing Route {route_count + 1}/{route_indexer.total} ===")
print(f"Route: {config.name}, Town: {config.town}", "Route_index: ", route_indexer.index)

### want route_id = 2509

route_id:  1711
route_id:  1773
route_id:  1790
Got the route_id:  1790

=== Processing Route 1/220 ===
Route: RouteScenario_1790, Town: Town12 Route_index:  3


In [12]:
# Step 1: Load scenario

crash_flag, save_name, entry_status, crash_message = my_load_scenario(leaderboard_evaluator, args, config)
print("crash_flag: ", crash_flag)
print("save_name: ", save_name)
print("entry_status: ", entry_status)
print("crash_message: ", crash_message)

save_name = save_name + '_' + model_name


========= Preparing RouteScenario_1790 (repetition 0) =========
> Loading the world
crash_flag:  False
save_name:  RouteScenario_1790_rep0_Town12_HazardAtSideLane_1_8_08_29_22_31_46
entry_status:  Started
crash_message:  


In [13]:
# Step 2: Load agent
if not crash_flag:
    crash_flag, entry_status, crash_message = my_load_agent(leaderboard_evaluator, args, config, save_name, entry_status, crash_message)

print("crash_flag: ", crash_flag)
print("entry_status: ", entry_status)
print("crash_message: ", crash_message)

> Setting up the agent
DummyAgent2 initialized - will save 10 images to /mnt3/Documents/AD_Framework/town_test_images
crash_flag:  False
entry_status:  Started
crash_message:  


In [14]:
# Step 3: Load route scenario
if not crash_flag:
    crash_flag, entry_status, crash_message = my_load_route_scenario(leaderboard_evaluator, args, config, entry_status, crash_message)

print("crash_flag: ", crash_flag)
print("entry_status: ", entry_status)
print("crash_message: ", crash_message)

> Running the route
crash_flag:  False
entry_status:  Started
crash_message:  


In [15]:
# Step 4: Setup scenario
if not crash_flag:
    my_run_scenario_setup(leaderboard_evaluator)

# Add image saving capability to the agent
import numpy as np
from datetime import datetime

# Monkey-patch the agent to capture sensor data
original_run_step = leaderboard_evaluator.agent_instance.run_step

def enhanced_run_step(input_data, timestamp):
    """Enhanced run_step that captures sensor data for image saving"""
    # Store the input data for later access
    leaderboard_evaluator.agent_instance.last_input_data = input_data
    leaderboard_evaluator.agent_instance.step = getattr(leaderboard_evaluator.agent_instance, 'step', 0) + 1
    
    # Call original run_step
    return original_run_step(input_data, timestamp)

# Replace the run_step method
leaderboard_evaluator.agent_instance.run_step = enhanced_run_step
leaderboard_evaluator.agent_instance.step = 0

print("Agent enhanced with image capture capability")

Agent enhanced with image capture capability


In [56]:
# Step 5: Run scenario loop step by step with automatic image saving
import cv2
import numpy as np
from pathlib import Path
from datetime import datetime

# Create save directory with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = Path(f"/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_{timestamp}")
save_dir.mkdir(parents=True, exist_ok=True)
print(f"Images will be saved to: {save_dir}")

def save_debug_images(agent_instance, save_dir, step_count):
    """Save all sensor images for debugging"""
    if not hasattr(agent_instance, 'last_input_data'):
        return
    
    input_data = agent_instance.last_input_data
    
    # Create step-specific subdirectory
    step_dir = save_dir / f"step_{step_count:04d}"
    step_dir.mkdir(exist_ok=True)
    
    # Save RGB cameras
    for camera_id in ['Center', 'Left', 'Right']:
        if camera_id in input_data:
            rgb_image = input_data[camera_id][1][:, :, :3]  # Remove alpha channel if present
            frame_num = input_data[camera_id][0]
            
            # Convert from RGB to BGR for OpenCV
            bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
            
            # Save image
            image_path = step_dir / f"{camera_id}_frame_{frame_num:06d}.jpg"
            cv2.imwrite(str(image_path), bgr_image)
            # print(f"  Saved: {camera_id} -> {image_path.name}")
    
    # Save sensor data info to text file
    info_path = step_dir / "sensor_info.txt"
    with open(info_path, 'w') as f:
        f.write(f"Step: {step_count}\n")
        f.write(f"Timestamp: {datetime.now()}\n\n")
        f.write("Sensor Data:\n")
        for key, val in input_data.items():
            if hasattr(val[1], 'shape'):
                f.write(f"  {key}: frame={val[0]:06d}, shape={val[1].shape}\n")
            else:
                f.write(f"  {key}: frame={val[0]:06d}\n")
    
    return step_dir

# Run simulation steps with image saving
run_step_n = 1  # run 5 steps = 0.25 second
total_steps_run = 0

if not crash_flag:
    print(f"\n=== Running {run_step_n} simulation steps with image capture ===")
    
    for step_i in range(run_step_n):
        # Run one step
        crash_flag_step, entry_status, crash_message = my_run_scenario_step(
            leaderboard_evaluator, entry_status, crash_message, n_steps=1
        )
        
        if crash_flag_step:
            crash_flag = True
            print(f"Crash detected at step {step_i}")
            break
        
        # Save images after each step
        if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
            print(f"\nStep {total_steps_run + step_i}:")
            step_dir = save_debug_images(
                leaderboard_evaluator.agent_instance, 
                save_dir, 
                total_steps_run + step_i
            )
        else:
            print(f"Step {total_steps_run + step_i}: No sensor data captured")
    
    total_steps_run += step_i + 1
    
print(f"\n=== Completed {total_steps_run} steps ===")
print(f"crash_flag: {crash_flag}")
print(f"entry_status: {entry_status}")
print(f"crash_message: {crash_message}")
print(f"Images saved to: {save_dir}")

# Display summary
if save_dir.exists():
    step_dirs = sorted(save_dir.glob("step_*"))
    print(f"\nSaved {len(step_dirs)} steps of data")
    for step_dir in step_dirs[:3]:  # Show first 3 steps
        images = list(step_dir.glob("*.jpg"))
        print(f"  {step_dir.name}: {len(images)} images")

Images will be saved to: /mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_20250829_224746

=== Running 1 simulation steps with image capture ===
In my_run_scenario_step (Check if it is stuck or not)
=== [Agent] -- Wallclock = 2025-08-29 22:47:46.050 -- System time = 907.450 -- Game time = 2.650 -- Ratio = 0.003x
Frame 53/10 - Test_0

Step 0:

=== Completed 1 steps ===
crash_flag: False
entry_status: Started
crash_message: 
Images saved to: /mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_20250829_224746

Saved 1 steps of data
  step_0000: 3 images


In [47]:
world_obj = leaderboard_evaluator.manager.scenario.world

In [55]:
actors = world_obj.get_actors()

# Filter only vehicles
vehicles = actors.filter("vehicle.*")

print("Total vehicles:", len(vehicles))
for v in vehicles:
    print(v.id, v.type_id, v.get_location())

    if str(v.id) == str(3763):
        get_transform_info(v)
        set_carla_loc_rot(v, 572.4432983398438, 3910.955322265625, 371.29156494140625, -179.70237731933594, -0.05358966067433357, -0.0012207030085846782)
       



Total vehicles: 13
3771 vehicle.lincoln.mkz_2020 Location(x=530.403687, y=3920.766357, z=371.202332)
3770 vehicle.mini.cooper_s_2021 Location(x=530.563171, y=3917.517334, z=371.213531)
3769 vehicle.ford.mustang Location(x=615.105164, y=3910.483154, z=371.281952)
3768 vehicle.lincoln.mkz_2017 Location(x=598.434021, y=3910.554932, z=371.336395)
3767 vehicle.dodge.charger_2020 Location(x=508.159088, y=3911.053955, z=371.199677)
3766 vehicle.ford.mustang Location(x=495.933777, y=3911.112061, z=371.119751)
3765 vehicle.ford.mustang Location(x=615.118042, y=3913.733398, z=371.281952)
3764 vehicle.mercedes.coupe_2020 Location(x=597.304993, y=3913.803711, z=371.303741)
3763 vehicle.chevrolet.impala Location(x=572.443298, y=3913.955322, z=371.291565)
transform.location.x 572.4432983398438
transform.location.y 3913.955322265625
transform.location.z 371.29156494140625
transform.rotation.yaw -179.70237731933594
transform.rotation.pitch -0.05358966067433357
transform.rotation.roll -0.00122070300858

In [ ]:
# 3763 vehicle.chevrolet.impala Location(x=572.704773, y=3913.956787, z=371.291656)

### Set position



### Move with object without pause the running scenarios

In [53]:
import carla

def get_transform_info(carla_vehicle_object):
    transform = carla_vehicle_object.get_transform()
    print("transform.location.x", transform.location.x)
    print("transform.location.y", transform.location.y)
    print("transform.location.z", transform.location.z)

    print("transform.rotation.yaw", transform.rotation.yaw)
    print("transform.rotation.pitch", transform.rotation.pitch)
    print("transform.rotation.roll", transform.rotation.roll)
    vel = carla_vehicle_object.get_velocity()
    print("vel.x", vel.x, "vel.y", vel.y, "vel.z", vel.z)
    return transform

def set_carla_loc_rot(carla_vehicle_object, x, y, z, yaw, pitch, roll):
    # Create a new location (x, y, z)
    loc = carla.Location(x=x, y=y, z=z)
    # Create a new rotation (pitch, yaw, roll)
    rot = carla.Rotation(pitch=pitch, yaw=yaw, roll=roll)
    new_transform = carla.Transform(loc, rot)
    carla_vehicle_object.set_transform(new_transform)

get_transform_info(leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0])
print("----")
get_transform_info(leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[1])
print("----")
get_transform_info(leaderboard_evaluator.manager.ego_vehicles[0])

transform.location.x 568.289306640625
transform.location.y 3914.0
transform.location.z 371.2752685546875
transform.rotation.yaw 179.71975708007812
transform.rotation.pitch -0.3596777319908142
transform.rotation.roll 0.0013611284084618092
vel.x -0.0007843188941478729 vel.y -7.83641153248027e-05 vel.z 0.2546674311161041
----
transform.location.x 528.2965698242188
transform.location.y 3910.070556640625
transform.location.z 371.2028503417969
transform.rotation.yaw 179.72560119628906
transform.rotation.pitch -0.21286283433437347
transform.rotation.roll -9.155279258266091e-05
vel.x -1.3914890587329865e-05 vel.y -0.00027825942379422486 vel.z 0.008414044976234436
----
transform.location.x 579.208984375
transform.location.y 3910.705322265625
transform.location.z 371.2976989746094
transform.rotation.yaw 179.7611541748047
transform.rotation.pitch 0.6786338686943054
transform.rotation.roll 0.0014464346459135413
vel.x -12.815984725952148 vel.y 0.05352751165628433 vel.z -0.01002237293869257


In [18]:
### move bike closer

set_carla_loc_rot(carla_vehicle_object=leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0], x=568.29, y=3914, z=371.2144, yaw=179.72, pitch=-0.458, roll=0)

In [ ]:
## relocate my car

set_carla_loc_rot(carla_vehicle_object=leaderboard_evaluator.manager.ego_vehicles[0], x=578.208984375, y=3910, z=371.29, yaw=179.76, pitch=0.708, roll=0)

In [21]:
### stop scenario manager

# print("leaderboard_evaluator.manager._scenario_thread.__dict__._is_stopped", leaderboard_evaluator.manager._scenario_thread._is_stopped)
print("leaderboard_evaluator.manager._running", leaderboard_evaluator.manager._running)
leaderboard_evaluator.manager._running = False
try:
    leaderboard_evaluator.manager._scenario_thread.join()
except:
    pass
leaderboard_evaluator.manager._scenario_thread = None
# print("leaderboard_evaluator.manager._scenario_thread.__dict__._is_stopped", leaderboard_evaluator.manager._scenario_thread._is_stopped)
print("leaderboard_evaluator.manager._running", leaderboard_evaluator.manager._running)

leaderboard_evaluator.manager._running True
leaderboard_evaluator.manager._running False


In [22]:
### start scenario manager

# leaderboard_evaluator.manager._tick_scenario()
leaderboard_evaluator.manager._running = True
# CarlaDataProvider.on_carla_tick()


In [24]:
### continue the previous thread

# def start_previous_loop_thread(leaderboard_evaluator_manager_obj):
#     # Thread for build_scenarios
#     leaderboard_evaluator_manager_obj._running = True
#     leaderboard_evaluator_manager_obj._scenario_thread = threading.Thread(target=leaderboard_evaluator_manager_obj.build_scenarios_loop, args=(leaderboard_evaluator_manager_obj._debug_mode > 0, ))
#     leaderboard_evaluator_manager_obj._scenario_thread.start()
#     print("leaderboard_evaluator.manager._scenario_thread.is_alive()", leaderboard_evaluator.manager._scenario_thread.is_alive())

#     if leaderboard_evaluator_manager_obj._running:
#         leaderboard_evaluator_manager_obj._tick_scenario()

def start_builder_thread(manager):
    if getattr(manager, "_scenario_thread", None) and manager._scenario_thread.is_alive():
        return  # already running
    manager._running = True
    manager._scenario_thread = threading.Thread(
        target=manager.build_scenarios_loop,
        args=(manager._debug_mode > 0,),
        daemon=True
    )
    manager._scenario_thread.start()

    if manager._running:
        manager._tick_scenario()

# start_previous_loop_thread(leaderboard_evaluator.manager)
start_builder_thread(leaderboard_evaluator.manager)


=== [Agent] -- Wallclock = 2025-08-29 22:34:15.245 -- System time = 96.645 -- Game time = 2.550 -- Ratio = 0.026x
Frame 51/10 - Test_0


In [ ]:
leaderboard_evaluator.manager._running

In [ ]:
leaderboard_evaluator.manager._scenario_thread.__dict__

In [ ]:
leaderboard_evaluator.manager._scenario_thread

In [ ]:
leaderboard_evaluator.manager.ego_vehicles[0].get_location().x, leaderboard_evaluator.manager.ego_vehicles[0].get_location().y, leaderboard_evaluator.manager.ego_vehicles[0].get_location().z

# leaderboard_evaluator.manager.ego_vehicles[0].attributes

In [ ]:
from carla import Location
new_loc = Location(x=589.208984375, y=3910.0215, z=371.29)

leaderboard_evaluator.manager.ego_vehicles[0].set_location(new_loc)

In [ ]:
leaderboard_evaluator

In [ ]:
leaderboard_evaluator.route_scenario.__class__

In [ ]:
leaderboard_evaluator.manager.scenario.__class__

In [ ]:
leaderboard_evaluator.route_scenario.list_scenarios[0].other_actors[0].attributes
# leaderboard_evaluator.route_scenario.occupied_parking_locations
# leaderboard_evaluator.route_scenario.available_parking_locations
# leaderboard_evaluator.route_scenario._parked_ids
# leaderboard_evaluator.route_scenario.ego_data

In [ ]:
leaderboard_evaluator.route_scenario.list_scenarios[0].other_actors[1].get_location().x

In [ ]:
leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0].get_location().x, leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0].get_location().y, leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0].get_location().z

In [ ]:
from carla import Location
new_loc = Location(x=568.2979, y=3914.0215, z=371.2144)

leaderboard_evaluator.manager.scenario.list_scenarios[0].other_actors[0].set_location(new_loc)


In [ ]:
leaderboard_evaluator.route_scenario.other_actors

In [ ]:
leaderboard_evaluator.manager.scenario.other_actors

In [ ]:
leaderboard_evaluator.manager.scenario.ego_vehicles

In [ ]:
leaderboard_evaluator.manager.scenario.build_scenarios(leaderboard_evaluator.manager.scenario.ego_vehicles)

In [ ]:
leaderboard_evaluator.manager.scenario.remove_all_actors()

In [ ]:
leaderboard_evaluator.manager.ego_vehicles[0].get_location().x

In [ ]:
leaderboard_evaluator.manager

In [ ]:
leaderboard_evaluator.manager.build_scenarios_loop(1)

In [ ]:
leaderboard_evaluator.agent_instance

In [ ]:
if hasattr(leaderboard_evaluator, 'world'):
    world = leaderboard_evaluator.world
    vehicles = world.get_actors().filter('vehicle.*')

vehicles

In [ ]:
# for v in vehicles:
#     print("v", v.attributes)


In [ ]:
leaderboard_evaluator.manager.scenario.list_scenarios
leaderboard_evaluator.manager.scenario.occupied_parking_locations
leaderboard_evaluator.manager.scenario.available_parking_locations
leaderboard_evaluator.manager.scenario._parked_ids
leaderboard_evaluator.manager.scenario.ego_data
leaderboard_evaluator.manager.scenario.behavior_node
leaderboard_evaluator.manager.scenario.other_actors








In [ ]:
leaderboard_evaluator.manager.scenario.list_scenarios[0].remove_all_actors()

In [ ]:
leaderboard_evaluator.route_scenario.other_actors

In [ ]:
# Import the necessary modules for snapshot functionality
import sys
import pickle
from pathlib import Path
from datetime import datetime
import copy

# Add the parent directory to path to import from carla_server
carla_server_path = '/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices'
if carla_server_path not in sys.path:
    sys.path.insert(0, carla_server_path)

# Import the WorldSnapshot class
from world_snapshot import WorldSnapshot

# Create a simple simulation state object to hold our snapshot
class SimState:
    def __init__(self):
        self.leaderboard_evaluator = None
        self.snapshots = {}
        self.snapshot_dir = Path("/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots")
        self.snapshot_dir.mkdir(parents=True, exist_ok=True)
        self.step_count = 0
        self.server_id = "notebook_test"
        self.last_observation = None

# Initialize simulation state
sim_state = SimState()
sim_state.leaderboard_evaluator = leaderboard_evaluator
sim_state.step_count = total_steps_run

print(f"Initialized sim_state with leaderboard_evaluator")
print(f"Snapshot directory: {sim_state.snapshot_dir}")
print(f"Current step count: {sim_state.step_count}")

In [ ]:
# Function to save a snapshot (adapted from carla_server.py)
def save_snapshot(snapshot_id=None):
    """Save current world state"""
    try:
        # Verify scenario manager exists
        if not hasattr(sim_state.leaderboard_evaluator, 'manager') or sim_state.leaderboard_evaluator.manager is None:
            raise ValueError("Scenario manager not initialized - cannot snapshot without manager")
        
        # Generate snapshot ID if not provided
        if snapshot_id is None:
            snapshot_id = f"snap_{sim_state.server_id}_{len(sim_state.snapshots)}_{datetime.now().strftime('%H%M%S')}"
        
        # Get world from leaderboard_evaluator
        world = None
        if hasattr(sim_state.leaderboard_evaluator, 'world'):
            world = sim_state.leaderboard_evaluator.world
            print(f"DEBUG: Got world from leaderboard_evaluator: {world}")
        else:
            print("ERROR: leaderboard_evaluator has no world attribute!")
        
        # Debug: Print available attributes
        print("DEBUG: leaderboard_evaluator attributes:")
        print(f"  - Has manager: {hasattr(sim_state.leaderboard_evaluator, 'manager')}")
        print(f"  - Has world: {hasattr(sim_state.leaderboard_evaluator, 'world')}")
        print(f"  - World object: {world}")
        print(f"  - Has agent_instance: {hasattr(sim_state.leaderboard_evaluator, 'agent_instance')}")
        print(f"  - Has route_scenario: {hasattr(sim_state.leaderboard_evaluator, 'route_scenario')}")
        
        # Get current observation if available
        current_observation = {}
        if hasattr(sim_state.leaderboard_evaluator.agent_instance, 'last_input_data'):
            input_data = sim_state.leaderboard_evaluator.agent_instance.last_input_data
            current_observation = {
                'images': {},
                'sensors': {}
            }
            # Store sensor data
            for key, val in input_data.items():
                if key in ['Center', 'Left', 'Right']:
                    current_observation['images'][key] = val[1]  # Store the image array
                else:
                    current_observation['sensors'][key] = val
            print(f"Captured observation with {len(current_observation['images'])} images")
        
        # Capture the snapshot
        phase_marker = f"Step {sim_state.step_count} - {snapshot_id}"
        snapshot = WorldSnapshot.capture(sim_state, world, phase_marker=phase_marker)
        snapshot.snapshot_id = snapshot_id
        
        # Store observation in snapshot
        snapshot.observation = current_observation
        
        # Store a deep copy of the snapshot
        sim_state.snapshots[snapshot_id] = copy.deepcopy(snapshot)
        
        # Save snapshot to disk
        snapshot_file = sim_state.snapshot_dir / f"{snapshot_id}.pkl"
        try:
            with open(snapshot_file, 'wb') as f:
                pickle.dump(snapshot, f)
            print(f"Snapshot saved to disk: {snapshot_file}")
        except Exception as e:
            print(f"Failed to save snapshot to disk: {e}")
        
        # Print snapshot details
        print(f"\\nSnapshot captured: {snapshot_id}")
        print(f"  - Vehicles: {len(snapshot.vehicles)}")
        print(f"  - Has scenario_manager state: {snapshot.scenario_manager is not None}")
        print(f"  - Saved to: {snapshot_file}")
        
        # Log ego vehicle position
        for vehicle_id, vehicle_state in snapshot.vehicles.items():
            if vehicle_state.is_hero:
                print(f"  - Ego position: x={vehicle_state.location['x']:.2f}, y={vehicle_state.location['y']:.2f}, z={vehicle_state.location['z']:.2f}")
                break
        
        if snapshot.scenario_manager:
            print(f"  - Manager tick: {snapshot.scenario_manager.tick_count}")
            print(f"  - Manager running: {snapshot.scenario_manager.running}")
            print(f"  - Ego vehicles: {len(snapshot.scenario_manager.ego_vehicle_ids)}")
        
        return snapshot_id
        
    except Exception as e:
        print(f"Error saving snapshot: {e}")
        import traceback
        traceback.print_exc()
        return None

# Save a snapshot of the current state
print("\\n=== Saving Snapshot of Current State ===")
snapshot_id = save_snapshot("test_snapshot_after_50_steps")

if snapshot_id:
    print(f"\\n✅ Successfully saved snapshot: {snapshot_id}")
    print(f"Total snapshots in memory: {len(sim_state.snapshots)}")
else:
    print("\\n❌ Failed to save snapshot")

In [ ]:
### Run for another 50 steps.

# Run another 50 simulation steps with image saving
print("\n" + "="*60)
print("Running Another 50 Steps with Image Capture")
print("="*60)

# Create new directory for this batch of images
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir_batch2 = Path(f"/mnt3/Documents/AD_Framework/bench2drive-gymnasium/bench2drive_microservices/notebooks/debug_snapshots/debug_images_batch2_{timestamp}")
save_dir_batch2.mkdir(parents=True, exist_ok=True)
print(f"Images will be saved to: {save_dir_batch2}")

run_step_n = 50  # Run 50 more steps
initial_step_count = total_steps_run

if not crash_flag:
    print(f"\n=== Running {run_step_n} simulation steps with image capture ===")
    print(f"Starting from step {total_steps_run}")
    
    for step_i in range(run_step_n):
        # Run one step
        crash_flag_step, entry_status, crash_message = my_run_scenario_step(
            leaderboard_evaluator, entry_status, crash_message, n_steps=1
        )
        
        if crash_flag_step:
            crash_flag = True
            print(f"Crash detected at step {step_i}")
            break
        
        # Save images after each step
        if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
            # Save images every 1 steps to reduce storage
            if step_i % 1 == 0:
                print(f"\nStep {total_steps_run + step_i}:")
                step_dir = save_debug_images(
                    leaderboard_evaluator.agent_instance, 
                    save_dir_batch2, 
                    total_steps_run + step_i
                )
                
                # Also log vehicle position
                if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
                    vehicle = leaderboard_evaluator.agent_instance._vehicle
                    transform = vehicle.get_transform()
                    print(f"  Vehicle position: x={transform.location.x:.2f}, y={transform.location.y:.2f}, z={transform.location.z:.2f}")
        else:
            if step_i % 10 == 0:
                print(f"Step {total_steps_run + step_i}: Running...")
    
    total_steps_run += step_i + 1
    
print(f"\n=== Completed {step_i + 1} additional steps ===")
print(f"Total steps run in session: {total_steps_run}")
print(f"crash_flag: {crash_flag}")
print(f"entry_status: {entry_status}")
print(f"crash_message: {crash_message}")
print(f"Images saved to: {save_dir_batch2}")

# Display summary
if save_dir_batch2.exists():
    step_dirs = sorted(save_dir_batch2.glob("step_*"))
    print(f"\nSaved {len(step_dirs)} sets of images")
    for step_dir in step_dirs[:3]:  # Show first 3 steps
        images = list(step_dir.glob("*.jpg"))
        print(f"  {step_dir.name}: {len(images)} images")

# Save a snapshot after these 50 steps
print("\n" + "="*60)
print("Saving Snapshot After Additional 50 Steps")
print("="*60)

snapshot_after_100 = save_snapshot(f"snapshot_after_{total_steps_run}_steps")
if snapshot_after_100:
    print(f"✅ Saved snapshot: {snapshot_after_100}")
    print(f"Total snapshots available: {len(sim_state.snapshots)}")
    
    # List all snapshots
    print("\nAll available snapshots:")
    for snap_id in sim_state.snapshots.keys():
        print(f"  - {snap_id}")

# Update sim_state step count
sim_state.step_count = total_steps_run

In [ ]:
### Restore snapshot 

# Restore to a previous snapshot and verify the state
print("\n" + "="*60)
print("Restoring to Previous Snapshot")
print("="*60)

# Display all available snapshots
print("\nAvailable snapshots to restore from:")
for i, snap_id in enumerate(sim_state.snapshots.keys()):
    snapshot = sim_state.snapshots[snap_id]
    print(f"  {i+1}. {snap_id}")
    print(f"     - Vehicles: {len(snapshot.vehicles)}")
    if hasattr(snapshot, 'scenario_manager') and snapshot.scenario_manager:
        print(f"     - Manager tick: {snapshot.scenario_manager.tick_count}")
    # Show ego position in snapshot  
    for vehicle_id, vehicle_state in snapshot.vehicles.items():
        if vehicle_state.is_hero:
            print(f"     - Ego position: x={vehicle_state.location['x']:.2f}, y={vehicle_state.location['y']:.2f}")
            break

# Choose which snapshot to restore (you can change this)
snapshot_to_restore = "test_snapshot_after_50_steps"  # Change this to any snapshot ID you want

print(f"\n>>> Restoring to: {snapshot_to_restore}")

# Get current state BEFORE restore
print("\n=== Current State Before Restore ===")
current_position = None
current_tick_count = None
if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
    vehicle = leaderboard_evaluator.agent_instance._vehicle
    transform = vehicle.get_transform()
    current_position = (transform.location.x, transform.location.y, transform.location.z)
    print(f"Current position: x={current_position[0]:.2f}, y={current_position[1]:.2f}, z={current_position[2]:.2f}")

if hasattr(leaderboard_evaluator, 'manager') and leaderboard_evaluator.manager:
    current_tick_count = leaderboard_evaluator.manager.tick_count
    print(f"Current manager tick count: {current_tick_count}")
    print(f"Current total_steps_run variable: {total_steps_run}")

# Save images BEFORE restore for comparison
print("\n=== Saving Images Before Restore ===")
timestamp_before = datetime.now().strftime("%Y%m%d_%H%M%S")
before_restore_dir = sim_state.snapshot_dir / f"before_restore_{timestamp_before}"
before_restore_dir.mkdir(parents=True, exist_ok=True)

if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
    save_debug_images(leaderboard_evaluator.agent_instance, before_restore_dir, current_tick_count if current_tick_count else total_steps_run)
    print(f"Saved 'before' images to: {before_restore_dir}")

# Perform the restore
try:
    print(f"\n=== Starting Restore of Snapshot: {snapshot_to_restore} ===")
    
    # Get the snapshot
    if snapshot_to_restore not in sim_state.snapshots:
        # Try to load from disk if not in memory
        snapshot_file = sim_state.snapshot_dir / f"{snapshot_to_restore}.pkl"
        if snapshot_file.exists():
            print(f"Loading snapshot from disk: {snapshot_file}")
            with open(snapshot_file, 'rb') as f:
                snapshot = pickle.load(f)
                sim_state.snapshots[snapshot_to_restore] = snapshot
        else:
            raise ValueError(f"Snapshot not found: {snapshot_to_restore}")
    
    snapshot = sim_state.snapshots[snapshot_to_restore]
    print(f"Found snapshot with {len(snapshot.vehicles)} vehicles")
    
    # Get the world
    world = sim_state.leaderboard_evaluator.world
    if world is None:
        raise ValueError("World is not available")
    
    # CRITICAL FIX: The restore method signature is def restore(self, sim_state, world)
    print("\nRestoring world state...")
    print(f"DEBUG: Calling restore with sim_state={sim_state}, world={world}")
    
    # Call restore with correct parameter order
    snapshot.restore(sim_state, world)
    
    # CRITICAL: Force the manager tick count to be restored
    if hasattr(snapshot, 'scenario_manager') and snapshot.scenario_manager:
        if hasattr(leaderboard_evaluator, 'manager') and leaderboard_evaluator.manager:
            restored_tick = snapshot.scenario_manager.tick_count
            leaderboard_evaluator.manager.tick_count = restored_tick
            print(f"\nForced manager tick count to: {restored_tick}")
            
            # Also update our tracking variables to match
            total_steps_run = restored_tick
            sim_state.step_count = restored_tick
            print(f"Updated total_steps_run to: {total_steps_run}")
    
    # CRITICAL FIX: Reinitialize watchdogs IMMEDIATELY after restore
    print("\n=== Fixing Watchdog Issues After Restore ===")
    from srunner.scenariomanager.watchdog import Watchdog
    
    if hasattr(leaderboard_evaluator, 'manager') and leaderboard_evaluator.manager:
        # Fix manager watchdog
        if not hasattr(leaderboard_evaluator.manager, '_watchdog') or leaderboard_evaluator.manager._watchdog is None:
            print("Reinitializing manager watchdog...")
            leaderboard_evaluator.manager._watchdog = Watchdog(leaderboard_evaluator.manager._timeout)
            leaderboard_evaluator.manager._watchdog.start()
            print("✅ Manager watchdog reinitialized")
        
        # Fix agent watchdog
        if not hasattr(leaderboard_evaluator.manager, '_agent_watchdog') or leaderboard_evaluator.manager._agent_watchdog is None:
            print("Reinitializing agent watchdog...")
            leaderboard_evaluator.manager._agent_watchdog = Watchdog(leaderboard_evaluator.manager._timeout)
            leaderboard_evaluator.manager._agent_watchdog.start()
            print("✅ Agent watchdog reinitialized")
    
    # Force a world tick to apply restored positions
    print("\nForcing world tick to apply restored positions...")
    if hasattr(leaderboard_evaluator, 'world') and leaderboard_evaluator.world:
        try:
            leaderboard_evaluator.world.tick()
            print("✅ World ticked successfully")
        except Exception as e:
            print(f"⚠️ Failed to tick world: {e}")
    
    # Verify ego position after restore
    print("\n=== State After Restore ===")
    if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
        vehicle = leaderboard_evaluator.agent_instance._vehicle
        transform = vehicle.get_transform()
        restored_position = (transform.location.x, transform.location.y, transform.location.z)
        print(f"Position AFTER restore: x={restored_position[0]:.2f}, y={restored_position[1]:.2f}, z={restored_position[2]:.2f}")
        
        # Calculate distance moved
        if current_position:
            distance = ((restored_position[0] - current_position[0])**2 + 
                       (restored_position[1] - current_position[1])**2 + 
                       (restored_position[2] - current_position[2])**2)**0.5
            print(f"Vehicle moved {distance:.2f} units during restore")
    
    if hasattr(leaderboard_evaluator, 'manager') and leaderboard_evaluator.manager:
        print(f"Manager tick count after restore: {leaderboard_evaluator.manager.tick_count}")
    
    # Restore observation if available and save restored images
    if hasattr(snapshot, 'observation') and snapshot.observation:
        sim_state.last_observation = snapshot.observation
        print(f"\nRestored observation with {len(snapshot.observation.get('images', {}))} images")
        
        # Save the restored images
        print("\n=== Saving Restored Images ===")
        timestamp_after = datetime.now().strftime("%Y%m%d_%H%M%S")
        after_restore_dir = sim_state.snapshot_dir / f"after_restore_{timestamp_after}"
        after_restore_dir.mkdir(parents=True, exist_ok=True)
        
        # Save each camera image from the restored observation
        for camera_id in ['Center', 'Left', 'Right']:
            if camera_id in snapshot.observation.get('images', {}):
                img_array = snapshot.observation['images'][camera_id]
                if len(img_array.shape) == 3:  # Make sure it's an image
                    # Convert and save
                    bgr_image = cv2.cvtColor(img_array[:, :, :3], cv2.COLOR_RGB2BGR)
                    image_path = after_restore_dir / f"restored_{camera_id}.jpg"
                    cv2.imwrite(str(image_path), bgr_image)
                    print(f"  Saved restored {camera_id} image to {image_path.name}")
        
        print(f"Restored images saved to: {after_restore_dir}")
    
    print(f"\n✅ Successfully restored snapshot: {snapshot_to_restore}")
    print(f"   Simulation state has been reset to tick {total_steps_run}")
    
except Exception as e:
    print(f"\n❌ Error restoring snapshot: {e}")
    import traceback
    traceback.print_exc()

# Run 50 steps after restore to fully demonstrate it's working
print("\n" + "="*60)
print("Running 50 Steps After Restore with Image Capture")
print("="*60)

timestamp_verify = datetime.now().strftime("%Y%m%d_%H%M%S")
verify_dir = sim_state.snapshot_dir / f"verify_after_restore_50steps_{timestamp_verify}"
verify_dir.mkdir(parents=True, exist_ok=True)

verification_steps = 50  # Run 50 steps to show proper restoration

# CRITICAL: Use the restored tick count as the starting point
restored_step_start = leaderboard_evaluator.manager.tick_count if hasattr(leaderboard_evaluator, 'manager') else total_steps_run

print(f"Starting from restored tick: {restored_step_start}")
print(f"Will run {verification_steps} steps and save images every step")
print(f"Expected to reach tick: {restored_step_start + verification_steps}\n")

for step_i in range(verification_steps):
    # Run one step
    crash_flag_step, entry_status, crash_message = my_run_scenario_step(
        leaderboard_evaluator, entry_status, crash_message, n_steps=1
    )
    
    if crash_flag_step:
        print(f"Crash detected at verification step {step_i}")
        break
    
    # Get current tick from manager
    current_tick = leaderboard_evaluator.manager.tick_count if hasattr(leaderboard_evaluator, 'manager') else (restored_step_start + step_i)
    
    # Save images every step to show progression
    if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
        print(f"\nTick {current_tick} (Verification step {step_i}):")
        step_dir = save_debug_images(
            leaderboard_evaluator.agent_instance, 
            verify_dir, 
            current_tick  # Use actual tick count, not step index
        )
        
        # Also log vehicle position
        if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
            vehicle = leaderboard_evaluator.agent_instance._vehicle
            transform = vehicle.get_transform()
            print(f"  Vehicle position: x={transform.location.x:.2f}, y={transform.location.y:.2f}, z={transform.location.z:.2f}")
    else:
        # Show progress
        if step_i % 10 == 0:
            print(f"Running tick {current_tick} (Verification step {step_i})...")

# Update total steps based on actual manager state
if hasattr(leaderboard_evaluator, 'manager'):
    total_steps_run = leaderboard_evaluator.manager.tick_count
else:
    total_steps_run = restored_step_start + step_i + 1

# Final position and summary
print(f"\n=== Completed {step_i + 1} verification steps ===")
print(f"Started from tick: {restored_step_start}")
print(f"Ended at tick: {total_steps_run}")
print(f"Total verification steps run: {step_i + 1}")

if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
    vehicle = leaderboard_evaluator.agent_instance._vehicle
    transform = vehicle.get_transform()
    print(f"\nFinal position after {verification_steps} steps: x={transform.location.x:.2f}, y={transform.location.y:.2f}, z={transform.location.z:.2f}")
    
    # Compare with original snapshot position
    for vehicle_id, vehicle_state in snapshot.vehicles.items():
        if vehicle_state.is_hero:
            original_x = vehicle_state.location['x']
            original_y = vehicle_state.location['y']
            distance_traveled = ((transform.location.x - original_x)**2 + 
                               (transform.location.y - original_y)**2)**0.5
            print(f"Distance traveled from restored point: {distance_traveled:.2f} units")
            break

print(f"\n✅ Successfully ran {step_i + 1} steps after restore!")
print(f"Verification images saved to: {verify_dir}")

# Count saved images and check their names
if verify_dir.exists():
    step_dirs = sorted(verify_dir.glob("step_*"))
    print(f"Saved {len(step_dirs)} sets of images")
    print("Image directories created:")
    for step_dir in step_dirs[:5]:  # Show first 5
        print(f"  - {step_dir.name}")
    if len(step_dirs) > 5:
        print(f"  ... and {len(step_dirs) - 5} more")

# Summary of all saved images
print("\n" + "="*60)
print("RESTORE VERIFICATION SUMMARY")
print("="*60)
print(f"1. Before restore images: {before_restore_dir}")
print(f"2. Restored snapshot images: {after_restore_dir if 'after_restore_dir' in locals() else 'Not saved (no observation in snapshot)'}")
print(f"3. After restore verification (50 steps): {verify_dir}")
print(f"\nThe simulation was restored to tick {restored_step_start} and ran to tick {total_steps_run}")
print(f"This confirms the restore {'WORKED - properly restored to step 50!' if restored_step_start == 50 else f'FAILED - started from tick {restored_step_start} instead of 50'}")

# CRITICAL: Check if the first image directory shows step 50 or 100
if verify_dir.exists():
    step_dirs = sorted(verify_dir.glob("step_*"))
    if step_dirs:
        first_dir = step_dirs[0].name
        print(f"\n🔍 CRITICAL CHECK - First verification image directory: {first_dir}")
        if "step_0050" in first_dir or "step_0051" in first_dir:
            print("✅ Images show restore WORKED - continuing from step 50!")
        elif "step_0100" in first_dir or "step_0101" in first_dir:
            print("❌ Images show restore FAILED - still at step 100!")
        else:
            print(f"⚠️ Unexpected step number in first directory: {first_dir}")
            
print("\n⚠️ IMPORTANT: Check the images in the verification directory to confirm:")
print(f"   {verify_dir}")
print("   The images should show the scenery from step 50, NOT step 100!")

# Optional: Save a new snapshot after these 50 verification steps
print("\n" + "="*60)
print("Saving New Snapshot After Verification")
print("="*60)
sim_state.step_count = total_steps_run
new_snapshot_id = save_snapshot(f"snapshot_after_restore_verification_{total_steps_run}_steps")
if new_snapshot_id:
    print(f"✅ Saved new snapshot: {new_snapshot_id}")
    print(f"Total snapshots available: {len(sim_state.snapshots)}")

In [ ]:
### Fix the restore by reinitializing watchdog and forcing tick

# After restore, we need to fix the watchdog issue and force a proper world tick
print("\n" + "="*60)
print("Fixing Restore Issues")
print("="*60)

# Fix 1: Reinitialize the watchdog that was set to None during restore
if hasattr(leaderboard_evaluator, 'manager') and leaderboard_evaluator.manager:
    from srunner.scenariomanager.watchdog import Watchdog
    
    # Recreate the watchdog if it's None
    if not hasattr(leaderboard_evaluator.manager, '_watchdog') or leaderboard_evaluator.manager._watchdog is None:
        print("Reinitializing manager watchdog...")
        leaderboard_evaluator.manager._watchdog = Watchdog(leaderboard_evaluator.manager._timeout)
        leaderboard_evaluator.manager._watchdog.start()
        print("✅ Watchdog reinitialized")
    
    # Also fix agent watchdog if needed
    if not hasattr(leaderboard_evaluator.manager, '_agent_watchdog') or leaderboard_evaluator.manager._agent_watchdog is None:
        print("Reinitializing agent watchdog...")
        leaderboard_evaluator.manager._agent_watchdog = Watchdog(leaderboard_evaluator.manager._timeout)
        leaderboard_evaluator.manager._agent_watchdog.start()
        print("✅ Agent watchdog reinitialized")

# Fix 2: Force a world tick to apply the restored positions
print("\nForcing world tick to apply restored positions...")
if hasattr(leaderboard_evaluator, 'world') and leaderboard_evaluator.world:
    try:
        # Force synchronous tick
        leaderboard_evaluator.world.tick()
        print("✅ World ticked successfully")
    except Exception as e:
        print(f"❌ Failed to tick world: {e}")

# Fix 3: Verify ego vehicle position after tick
if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
    vehicle = leaderboard_evaluator.agent_instance._vehicle
    transform = vehicle.get_transform()
    print(f"\nEgo position after fixes: x={transform.location.x:.2f}, y={transform.location.y:.2f}, z={transform.location.z:.2f}")
    
    # Check if we're at the right position now
    expected_x = 580.47  # From snapshot
    expected_y = 3910.70
    distance_from_expected = ((transform.location.x - expected_x)**2 + (transform.location.y - expected_y)**2)**0.5
    if distance_from_expected < 5.0:  # Within 5 meters
        print(f"✅ Position restored correctly! (within {distance_from_expected:.2f}m of expected)")
    else:
        print(f"⚠️ Position still incorrect! {distance_from_expected:.2f}m from expected position")
        print(f"   Expected: x={expected_x:.2f}, y={expected_y:.2f}")
        print(f"   Got: x={transform.location.x:.2f}, y={transform.location.y:.2f}")

print("\nRestore fixes applied. Ready to continue simulation.")

In [ ]:
### Run verification steps after fixing the restore

print("\n" + "="*60)
print("Running Verification After Restore Fixes")
print("="*60)

timestamp_verify = datetime.now().strftime("%Y%m%d_%H%M%S")
verify_dir = sim_state.snapshot_dir / f"verify_after_fix_{timestamp_verify}"
verify_dir.mkdir(parents=True, exist_ok=True)

verification_steps = 50
restored_step_start = leaderboard_evaluator.manager.tick_count if hasattr(leaderboard_evaluator, 'manager') else 50

print(f"Starting from tick: {restored_step_start}")
print(f"Will run {verification_steps} steps and save images every 5 steps")
print(f"Images will be saved to: {verify_dir}\n")

for step_i in range(verification_steps):
    # Run one step
    crash_flag_step, entry_status, crash_message = my_run_scenario_step(
        leaderboard_evaluator, entry_status, crash_message, n_steps=1
    )
    
    if crash_flag_step:
        print(f"Crash detected at verification step {step_i}")
        break
    
    # Get current tick from manager
    current_tick = leaderboard_evaluator.manager.tick_count if hasattr(leaderboard_evaluator, 'manager') else (restored_step_start + step_i)
    
    # Save images every 5 steps
    if step_i % 5 == 0:
        if hasattr(leaderboard_evaluator.agent_instance, 'last_input_data'):
            print(f"\nTick {current_tick} (Step {step_i}):")
            step_dir = save_debug_images(
                leaderboard_evaluator.agent_instance, 
                verify_dir, 
                current_tick
            )
            
            # Log vehicle position
            if hasattr(leaderboard_evaluator.agent_instance, '_vehicle') and leaderboard_evaluator.agent_instance._vehicle:
                vehicle = leaderboard_evaluator.agent_instance._vehicle
                transform = vehicle.get_transform()
                print(f"  Vehicle at: x={transform.location.x:.2f}, y={transform.location.y:.2f}, z={transform.location.z:.2f}")

# Update total steps
if hasattr(leaderboard_evaluator, 'manager'):
    total_steps_run = leaderboard_evaluator.manager.tick_count
else:
    total_steps_run = restored_step_start + step_i + 1

print(f"\n=== Verification Complete ===")
print(f"Ran {step_i + 1} steps successfully")
print(f"Final tick: {total_steps_run}")
print(f"Images saved to: {verify_dir}")

# Check the saved images
if verify_dir.exists():
    step_dirs = sorted(verify_dir.glob("step_*"))
    print(f"\nSaved {len(step_dirs)} sets of images:")
    for step_dir in step_dirs[:3]:
        print(f"  - {step_dir.name}")
    
    # CRITICAL: Check if the first image directory shows step 50 or 100
    if step_dirs:
        first_dir = step_dirs[0].name
        print(f"\n🔍 First image directory: {first_dir}")
        if "step_0050" in first_dir or "step_0051" in first_dir:
            print("✅ Images show restore WORKED - continuing from step 50!")
        elif "step_0100" in first_dir or "step_0101" in first_dir:
            print("❌ Images show restore FAILED - still at step 100!")
        else:
            print(f"⚠️ Unexpected step number in first directory: {first_dir}")

print("\n" + "="*60)
print("FINAL VERIFICATION")
print("="*60)
print("Check these directories to verify the restore worked:")
print(f"1. {verify_dir}/step_{restored_step_start:04d} - Should show scenery from step 50")
print(f"2. Compare with original step 50 images")
print("\nIf the images match the original step 50 scenery, the restore worked!")
print("If they show step 100 scenery, the restore failed to actually reset the world state.")